In [1]:
from gurobipy import *
import networkx as nx
import math
import re
import pandas as pd

In [2]:
def data_load(fileName):
    listSet = []
    list_temp = []
    start_flag = False
    with open(fileName, "r") as file:
        for line in file:
            if line.find('Days') > -1:
                days = int(line.split(":")[1])
            if line.find('Periods_per_day') > -1:
                periods = int(line.split(":")[1])
            if line.strip() == "":
                if list_temp:
                    listSet.append(list_temp.copy())
                    # print(list_temp)
                    list_temp.clear()
                start_flag = True
                continue
            if start_flag == True:
                if line.find(":") > -1:
                    continue
                else:
                    list_temp.append(line.split())

    df_courses = pd.DataFrame(listSet[0], columns=['courseID','teacher','countOfLectures','minWorkingDays','countOfStudents'])
    df_courses = df_courses.set_index('courseID', drop=True)
    df_courses[['countOfLectures']] = df_courses[['countOfLectures']].astype(int)
    df_courses[['minWorkingDays']] = df_courses[['minWorkingDays']].astype(int)
    df_courses[['countOfStudents']] = df_courses[['countOfStudents']].astype(int)

    df_rooms = pd.DataFrame(listSet[1], columns=['roomID', 'capacity'])
    df_rooms = df_rooms.set_index('roomID', drop=True)
    df_rooms[['capacity']] = df_rooms[['capacity']].astype(int)

    dict_curricula = {}
    df_unavailability_constraints = pd.DataFrame(listSet[3], columns=['courseID','day','dayPeriod'])
    df_unavailability_constraints = df_unavailability_constraints.set_index('courseID', drop=True)
    df_unavailability_constraints[['day']] = df_unavailability_constraints[['day']].astype(int)
    df_unavailability_constraints[['dayPeriod']] = df_unavailability_constraints[['dayPeriod']].astype(int)

    dict_temp = {}
    for item in listSet[2]:
        for i in range(len(item)):
            if i == 1:
                dict_temp['count'] = int(item[i])
            if i == 2:
                dict_temp['members'] = item[i:]
        dict_curricula[item[0]] = dict_temp.copy()
        dict_temp.clear()

    return df_courses, df_rooms, dict_curricula, df_unavailability_constraints, days, periods

In [3]:
def set2list(init_set):
    result_list = []
    temp_list = list(init_set)
    for i in range(len(temp_list) - 1):
        for j in temp_list[i + 1:]:
            result_list.append({temp_list[i], j})
    return result_list

In [7]:
df_courses, df_rooms, dict_curricula, df_unavailability_constraints, days, periods = data_load('comp02.ctt')

# courses K
courses = df_courses.index.to_list()

# rooms R
rooms = df_rooms.index.to_list()

# curriculas Curriculas
curriculas = dict_curricula.keys()

# time_slots T
time_slots = []
for i in range(days):
    for j in range(periods):
        time_slots.append((i,j))

# teachers Teachers
teachers = df_courses['teacher'].unique()


# # for course in which curricula
# courses_curriculas_pair = {}
# temp_set = set()
# for course in courses:
#     for curricula in curriculas:
#         if course in dict_curricula[curricula]['members']:
#             temp_set.add(curricula)
#     courses_curriculas_pair[course] = temp_set.copy()
#     temp_set.clear()

temp_list = []
temp = set()
curriculas_courses_pair = {}
for curricula in curriculas:
    for course in dict_curricula[curricula]['members']:
        temp.add((curricula,course))


temp

[('q000', 'c0129'),
 ('q000', 'c0131'),
 ('q000', 'c0152'),
 ('q000', 'c0157'),
 ('q001', 'c0129'),
 ('q001', 'c0131'),
 ('q001', 'c0152'),
 ('q001', 'c0155'),
 ('q002', 'c0129'),
 ('q002', 'c0131'),
 ('q002', 'c0152'),
 ('q002', 'c0156'),
 ('q003', 'c0211'),
 ('q003', 'c0213'),
 ('q003', 'c0238'),
 ('q004', 'c0230'),
 ('q004', 'c0023'),
 ('q004', 'c0217'),
 ('q005', 'c0007'),
 ('q005', 'c0009'),
 ('q005', 'c0230'),
 ('q005', 'c0023'),
 ('q006', 'c0019'),
 ('q006', 'c0310'),
 ('q006', 'c0088'),
 ('q006', 'c0080'),
 ('q007', 'c0015'),
 ('q007', 'c0019'),
 ('q007', 'c0020'),
 ('q007', 'c0368'),
 ('q009', 'c0217'),
 ('q009', 'c0219'),
 ('q009', 'c0023'),
 ('q009', 'c0230'),
 ('q010', 'c0310'),
 ('q010', 'c0633'),
 ('q010', 'c0302'),
 ('q010', 'c0088'),
 ('q011', 'c0310'),
 ('q011', 'c0633'),
 ('q011', 'c0298'),
 ('q012', 'c0044'),
 ('q012', 'c0056'),
 ('q012', 'c0069'),
 ('q013', 'c0266'),
 ('q013', 'c0054'),
 ('q013', 'c0058'),
 ('q014', 'c0323'),
 ('q014', 'c0061'),
 ('q014', 'c0327'),
